In [1]:
!pip install contractions
!pip install emoji --upgrade
!pip install transformers
!pip install spellchecker
!pip install pyspellchecker
!pip install germansentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for spellchecker: filename=spellchecker-0.4-py3-none-any.whl size=3966498 sha256=d64a8daa56fc63f7881320198de79a5e55e96c00660fd232fefb656edfc6426c
  Stored in directory: /root/.cache/pip/wheels/6c/90/c3/eac248d8755b2a7343487a2087b4b29ad98f388c3c8c69c286
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7122 sha256=5752f4447f054b2451fe36193b3d8da738a590a6842d06ac901c6e4dad4c6dcc
  Stored in directory: /root/.cache/pip/wheels/63/19/2c/5e9f447f2533d457a1167c3

In [2]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk
from bokeh.io import output_notebook
output_notebook()
from pathlib import Path


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import BertTokenizer
from germansentiment import SentimentModel
from sklearn.metrics import confusion_matrix, classification_report
from transformers import pipeline

In [4]:
# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Initialize the stemmer and stopwords for German
stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))

def clean_text(text, for_embedding=False):
    """
    - Remove any HTML tags (< /br> often found).
    - Keep only ASCII + European Chars and whitespace, no digits.
    - Remove single letter chars.
    - Convert all whitespaces (tabs etc.) to a single whitespace.
    - Remove URLs.
    - If for_embedding (True):
        - No lowercase.
        - Remove punctuation.
        - No stopwords removal and no lemmatization.
    - If not for_embedding (False):
        - Apply stemming.
        - No lowercase.
        - Remove stopwords and punctuation.
    """
    RE_WSPACE = re.compile(r"\s+")
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_URLS = re.compile(r"http[s]?://\S+|www\.\S+")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]")
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b")
    RE_RETWEET = re.compile(r"RT\s+")

    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]")
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b")

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_URLS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
    text = re.sub(RE_RETWEET, "", text)

    word_tokens = word_tokenize(text)
    words_filtered = []

    if for_embedding:
        words_filtered = word_tokens
    else:
        for word in word_tokens:
            if word not in stop_words:
                stemmed_word = stemmer.stem(word)
                words_filtered.append(stemmed_word)

    text_clean = " ".join(words_filtered)
    return text_clean


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **germeval2017 DataSet**

In [6]:
df_combined = pd.read_csv("/content/drive/MyDrive/Data Set/train_v1.4.tsv.txt", delimiter='\t')
df_combined.columns =['f1', 'text', 'f3', 'sentiment','f5']
df_combined

,f1,text,f3,sentiment,f5
0,http://twitter.com/MrElch/statuses/68737328126...,@nordschaf theoretisch kannste dir überall im ...,True,positive,Zugfahrt#Streckennetz:positive Zugfahrt#Fahrtz...
1,http://twitter.com/wolfi\_wiese/statuses/64924...,Bahn verspätet sich..gleich kommt noch jemand ...,True,negative,Zugfahrt#Pünktlichkeit:negative
2,http://wirtschaftsnachrichten-online.de/2015/1...,Ihre Anfragen brachten uns zu neuen Leistungen...,False,neutral,NaN
3,http://community.bahn.de/questions/1034649-db-...,Kann ich mit dem DB Geschenk Ticket den ICE Sp...,True,neutral,Allgemein#Haupt:neutral
4,http://www.facebook.com/220990034695139/posts/...,"Re: Frank Hennemann ""Tolle Veranstaltung der C...",True,neutral,Allgemein#Haupt:neutral
...,...,...,...,...,...
20935,http://twitter.com/annotin/statuses/6606183479...,"Schmierfilmbildung auf Schienen, S-Bahn Verspä...",True,negative,Sonstige_Unregelmässigkeiten#Haupt:negative Zu...
20936,http://www.facebook.com/160435867330906/posts/...,Re: NPD Landesverband Bayern Ja richtig so ! A...,True,negative,Atmosphäre#Haupt:negative
20937,http://twitter.com/sollnitservice/statuses/718...,https://t.co/C3idFnIoAf Günstige Bahn-Tickets:...,True,neutral,Allgemein#Haupt:neutral
20938,http://www.gutefrage.net/frage/ich-darf-mit-16...,"Ich darf mit 16 nicht alleine Zug fahren, was ...",True,neutral,Allgemein#Haupt:neutral


In [7]:
df = df_combined.fillna("")
df['text']
df['cleaned_text'] =df['text'].apply(clean_text)

In [8]:
df.drop(133,inplace = True)
df.reset_index(drop=True, inplace=True)

In [9]:
df_final = df[['text', 'sentiment', 'cleaned_text']].iloc[0:200]
df_final

,text,sentiment,cleaned_text
0,@nordschaf theoretisch kannste dir überall im ...,positive,nordschaf theoret kann uberall koln stadtberei...
1,Bahn verspätet sich..gleich kommt noch jemand ...,negative,bahn verspatet gleich kommt jemand druckt ries...
2,Ihre Anfragen brachten uns zu neuen Leistungen...,neutral,ihr anfrag bracht neu leistung wirtschaftsnach...
3,Kann ich mit dem DB Geschenk Ticket den ICE Sp...,neutral,kann db geschenk ticket ice sprint nutz sind i...
4,"Re: Frank Hennemann ""Tolle Veranstaltung der C...",neutral,re frank hennemann toll veranstalt cdu heut st...
...,...,...,...
195,Diese gar nicht nationalistische Band spielt n...,neutral,dies gar nationalist band spielt nachh oberhau...
196,#Job #Stellenangebot http://t.co/Sd6jUJnCsg - ...,neutral,job stellenangebot deutsch bahn ag quereinstie...
197,Deutsche Bahn investiert weiter massiv in die ...,neutral,deutsch bahn investiert massiv infrastruktur s...
198,"Die Bahn wird, laut Aussage Grubes, in Zukunft...",neutral,die bahn laut aussag grub zukunft flott fahrer...


In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
s = 0
indices_to_remove=[]
for i, long_text in enumerate(df_final['cleaned_text']):
    t = tokenizer.tokenize(long_text)
    #print(len(t))
    if len(t) > 512:
      #print(len(t))
      indices_to_remove.append(i)
      s=s+1
print(indices_to_remove)
#print(s)
# Remove rows from the DataFrame
#df_final['cleaned_text']=df_final['cleaned_text'].drop(indices_to_remove)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

[2, 9, 11, 40, 73, 108, 112, 133, 142, 176, 194]


In [11]:
df_final = df_final.drop(indices_to_remove)

In [12]:
df_final.reset_index(drop=True, inplace=True)
df_final

,text,sentiment,cleaned_text
0,@nordschaf theoretisch kannste dir überall im ...,positive,nordschaf theoret kann uberall koln stadtberei...
1,Bahn verspätet sich..gleich kommt noch jemand ...,negative,bahn verspatet gleich kommt jemand druckt ries...
2,Kann ich mit dem DB Geschenk Ticket den ICE Sp...,neutral,kann db geschenk ticket ice sprint nutz sind i...
3,"Re: Frank Hennemann ""Tolle Veranstaltung der C...",neutral,re frank hennemann toll veranstalt cdu heut st...
4,"RT @KleineHyaene: ""Mama, der bei der Bahn war ...",negative,kleinehya mama bahn inkompetent beim rausgeh m...
...,...,...,...
184,Diese gar nicht nationalistische Band spielt n...,neutral,dies gar nationalist band spielt nachh oberhau...
185,#Job #Stellenangebot http://t.co/Sd6jUJnCsg - ...,neutral,job stellenangebot deutsch bahn ag quereinstie...
186,Deutsche Bahn investiert weiter massiv in die ...,neutral,deutsch bahn investiert massiv infrastruktur s...
187,"Die Bahn wird, laut Aussage Grubes, in Zukunft...",neutral,die bahn laut aussag grub zukunft flott fahrer...


In [13]:
model = SentimentModel()
texts = df_final['cleaned_text']
result = model.predict_sentiment(texts)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
merged_df = pd.DataFrame(columns = ['Text','Label'])
merged_df['Text'] = df_final['cleaned_text']
merged_df['Label'] = result

In [15]:
# true labels
y_true = df_final['sentiment']

# predicted labels
#y_pred = result
y_pred=merged_df['Label']

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[27 18  1]
 [29 97  0]
 [ 7  9  1]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.43      0.59      0.50        46
     neutral       0.78      0.77      0.78       126
    positive       0.50      0.06      0.11        17

    accuracy                           0.66       189
   macro avg       0.57      0.47      0.46       189
weighted avg       0.67      0.66      0.65       189



In [17]:
model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

sentiment_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path)
result1 =[]
for l in df_final['cleaned_text']:
  result1.append(sentiment_classifier(l, truncation=True, max_length=512))

result1

[[{'label': 'Neutral', 'score': 0.9913060069084167}],
 [{'label': 'Neutral', 'score': 0.9587326645851135}],
 [{'label': 'Neutral', 'score': 0.9905479550361633}],
 [{'label': 'Neutral', 'score': 0.9861721396446228}],
 [{'label': 'Negative', 'score': 0.69915771484375}],
 [{'label': 'Neutral', 'score': 0.9913172721862793}],
 [{'label': 'Neutral', 'score': 0.9869369864463806}],
 [{'label': 'Neutral', 'score': 0.9751758575439453}],
 [{'label': 'Neutral', 'score': 0.9873479604721069}],
 [{'label': 'Neutral', 'score': 0.9898067116737366}],
 [{'label': 'Neutral', 'score': 0.9833337664604187}],
 [{'label': 'Neutral', 'score': 0.987196683883667}],
 [{'label': 'Neutral', 'score': 0.9727516770362854}],
 [{'label': 'Neutral', 'score': 0.9912795424461365}],
 [{'label': 'Neutral', 'score': 0.9821106791496277}],
 [{'label': 'Neutral', 'score': 0.9911069273948669}],
 [{'label': 'Neutral', 'score': 0.9899010062217712}],
 [{'label': 'Neutral', 'score': 0.9903898239135742}],
 [{'label': 'Neutral', 'score'

In [18]:
labels_sent = [d['label'] for lst2 in result1 for d in lst2]
df_senti_model=pd.DataFrame(labels_sent)
df_senti_model[0:]

,0
0,Neutral
1,Neutral
2,Neutral
3,Neutral
4,Negative
...,...
184,Neutral
185,Neutral
186,Neutral
187,Neutral


In [19]:
y_true = df_final['sentiment']
result_for_conf = df_senti_model.assign(C=y_true)
df_confusion1 = result_for_conf.applymap(lambda s: s.lower() if type(s) == str else s)
df_confusion1

,0,C
0,neutral,positive
1,neutral,negative
2,neutral,neutral
3,neutral,neutral
4,negative,negative
...,...,...
184,neutral,neutral
185,neutral,neutral
186,neutral,neutral
187,neutral,neutral


In [20]:
# true labels
y_true = df_confusion1.iloc[:,1]
# predicted labels
y_pred = df_confusion1.iloc[:,0]

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[  4  42   0]
 [  4 121   1]
 [  1  16   0]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.44      0.09      0.15        46
     neutral       0.68      0.96      0.79       126
    positive       0.00      0.00      0.00        17

    accuracy                           0.66       189
   macro avg       0.37      0.35      0.31       189
weighted avg       0.56      0.66      0.56       189



# **Scare DataSet**

In [21]:
df_combined = pd.read_csv("/content/drive/MyDrive/Data Set/scare_v1.0.0/scare_v1.0.0/annotations/alarm_clocks.csv", delimiter='\t')
df_combined.columns =['f1','f2','f3','f4', 'text', 'f3', 'sentiment','f5']
df_combined

,f1,f2,f3,f4,text,f3,sentiment,f5
0,aspect,7000,21,27,Update,7000-aspect1,Neutral,Related
1,subjective,7000,28,40,funktioniert,7000-subjective1,Positive,Related
2,subjective,7001,0,10,Echt super,7001-subjective5,Positive,Related
3,subjective,7001,15,22,Schönes,7001-subjective4,Positive,Related
4,subjective,7001,38,51,einzigartiges,7001-subjective3,Positive,Related
...,...,...,...,...,...,...,...,...
642,subjective,7158,0,5,Super,7158-subjective0,Positive,Related
643,subjective,7159,49,62,nur empfehlen,7159-subjective2,Positive,Related
644,aspect,7159,33,39,wecker,7159-aspect0,Neutral,Related
645,subjective,7159,21,32,echt toller,7159-subjective1,Positive,Related


In [22]:
filtered_df = df_combined[df_combined['sentiment'] != 'Unknown'].reset_index(drop=True)
filtered_df

,f1,f2,f3,f4,text,f3,sentiment,f5
0,aspect,7000,21,27,Update,7000-aspect1,Neutral,Related
1,subjective,7000,28,40,funktioniert,7000-subjective1,Positive,Related
2,subjective,7001,0,10,Echt super,7001-subjective5,Positive,Related
3,subjective,7001,15,22,Schönes,7001-subjective4,Positive,Related
4,subjective,7001,38,51,einzigartiges,7001-subjective3,Positive,Related
...,...,...,...,...,...,...,...,...
594,subjective,7158,0,5,Super,7158-subjective0,Positive,Related
595,subjective,7159,49,62,nur empfehlen,7159-subjective2,Positive,Related
596,aspect,7159,33,39,wecker,7159-aspect0,Neutral,Related
597,subjective,7159,21,32,echt toller,7159-subjective1,Positive,Related


In [23]:
df = filtered_df.fillna("")
df['text']
df['cleaned_text'] =df['text'].apply(clean_text)

In [24]:
model = SentimentModel()
texts = df['cleaned_text']
result = model.predict_sentiment(texts)

In [25]:
merged_df = pd.DataFrame(columns = ['Text','Label'])
merged_df['Text'] = df['cleaned_text']
merged_df['Label'] = result

In [26]:
sentiment_mapping = {'positive': 'Positive', 'negative': 'Negative', 'neutral': 'Neutral'}
filtered_df['sentiment'] = filtered_df['sentiment'].replace(sentiment_mapping)
merged_df['Label']=merged_df['Label'].replace(sentiment_mapping)

In [27]:
# true labels
y_true = filtered_df['sentiment']

# predicted labels
#y_pred = result
y_pred=merged_df['Label']

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[ 58   0  20]
 [ 52   4 157]
 [ 32   2 274]]
Classification Report:
              precision    recall  f1-score   support

    Negative       0.41      0.74      0.53        78
     Neutral       0.67      0.02      0.04       213
    Positive       0.61      0.89      0.72       308

    accuracy                           0.56       599
   macro avg       0.56      0.55      0.43       599
weighted avg       0.60      0.56      0.45       599



In [28]:
model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

sentiment_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path)
result1 =[]
for l in df['cleaned_text']:
  result1.append(sentiment_classifier(l))

result1


[[{'label': 'Neutral', 'score': 0.8813788890838623}],
 [{'label': 'Neutral', 'score': 0.8788792490959167}],
 [{'label': 'Neutral', 'score': 0.7416722178459167}],
 [{'label': 'Neutral', 'score': 0.9688530564308167}],
 [{'label': 'Positive', 'score': 0.6927860975265503}],
 [{'label': 'Neutral', 'score': 0.9900013208389282}],
 [{'label': 'Positive', 'score': 0.9845167994499207}],
 [{'label': 'Neutral', 'score': 0.9857368469238281}],
 [{'label': 'Neutral', 'score': 0.9899167418479919}],
 [{'label': 'Positive', 'score': 0.9902946949005127}],
 [{'label': 'Neutral', 'score': 0.9825902581214905}],
 [{'label': 'Neutral', 'score': 0.9696903824806213}],
 [{'label': 'Neutral', 'score': 0.9891749620437622}],
 [{'label': 'Positive', 'score': 0.9782214760780334}],
 [{'label': 'Neutral', 'score': 0.9602766633033752}],
 [{'label': 'Neutral', 'score': 0.9907517433166504}],
 [{'label': 'Neutral', 'score': 0.8389200568199158}],
 [{'label': 'Neutral', 'score': 0.9926215410232544}],
 [{'label': 'Neutral', '

In [29]:
labels_sent = [d['label'] for lst2 in result1 for d in lst2]
df_senti_model=pd.DataFrame(labels_sent)
df_senti_model[0:]

,0
0,Neutral
1,Neutral
2,Neutral
3,Neutral
4,Positive
...,...
594,Neutral
595,Neutral
596,Neutral
597,Positive


In [30]:
y_true = filtered_df['sentiment']
result_for_conf = df_senti_model.assign(C=y_true)
df_confusion1 = result_for_conf.applymap(lambda s: s.lower() if type(s) == str else s)
df_confusion1


,0,C
0,neutral,neutral
1,neutral,positive
2,neutral,positive
3,neutral,positive
4,positive,positive
...,...,...
594,neutral,positive
595,neutral,positive
596,neutral,neutral
597,positive,positive


In [31]:
# true labels
y_true = df_confusion1.iloc[:,1]
# predicted labels
y_pred = df_confusion1.iloc[:,0]

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[ 12  62   4]
 [  1 209   3]
 [  6 177 125]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.63      0.15      0.25        78
     neutral       0.47      0.98      0.63       213
    positive       0.95      0.41      0.57       308

    accuracy                           0.58       599
   macro avg       0.68      0.51      0.48       599
weighted avg       0.74      0.58      0.55       599



# **Scare Fitness Tracker DataSet**

In [32]:
df_combined = pd.read_csv("/content/drive/MyDrive/Data Set/scare_v1.0.0/scare_v1.0.0/annotations/fitness_tracker.csv", delimiter='\t')
df_combined.columns =['f1','f2','f3','f4', 'text', 'f3', 'sentiment','f5']
df_combined

,f1,f2,f3,f4,text,f3,sentiment,f5
0,aspect,1000,48,55,pro app,1000-aspect2,Neutral,Related
1,subjective,1000,80,102,immer noch nicht genug,1000-subjective4,Negative,Related
2,aspect,1000,123,139,Kauf des coaches,1000-aspect1,Neutral,Related
3,subjective,1000,140,149,gezwungen,1000-subjective3,Negative,Related
4,subjective,1000,185,205,ziemlich unverschämt,1000-subjective2,Negative,Related
...,...,...,...,...,...,...,...,...
566,aspect,1157,25,35,Freeletics,1157-aspect1,Neutral,Related
567,aspect,1157,10,21,Fitness App,1157-aspect0,Neutral,Related
568,subjective,1157,4,9,beste,1157-subjective0,Positive,Related
569,subjective,1158,0,5,Super,1158-subjective0,Positive,Related


In [33]:
filtered_df = df_combined[df_combined['sentiment'] != 'Unknown'].reset_index(drop=True)
filtered_df

,f1,f2,f3,f4,text,f3,sentiment,f5
0,aspect,1000,48,55,pro app,1000-aspect2,Neutral,Related
1,subjective,1000,80,102,immer noch nicht genug,1000-subjective4,Negative,Related
2,aspect,1000,123,139,Kauf des coaches,1000-aspect1,Neutral,Related
3,subjective,1000,140,149,gezwungen,1000-subjective3,Negative,Related
4,subjective,1000,185,205,ziemlich unverschämt,1000-subjective2,Negative,Related
...,...,...,...,...,...,...,...,...
516,aspect,1157,25,35,Freeletics,1157-aspect1,Neutral,Related
517,aspect,1157,10,21,Fitness App,1157-aspect0,Neutral,Related
518,subjective,1157,4,9,beste,1157-subjective0,Positive,Related
519,subjective,1158,0,5,Super,1158-subjective0,Positive,Related


In [34]:
df = filtered_df.fillna("")
df['text']
df['cleaned_text'] =df['text'].apply(clean_text)

In [35]:
model = SentimentModel()
texts = df['cleaned_text']
result = model.predict_sentiment(texts)

In [36]:
merged_df = pd.DataFrame(columns = ['Text','Label'])
merged_df['Text'] = df['cleaned_text']
merged_df['Label'] = result

# Print the merged data frame
merged_df

,Text,Label
0,pro app,negative
1,imm genug,positive
2,kauf coach,positive
3,gezwung,negative
4,ziemlich unverschamt,negative
...,...,...
516,freeletics,positive
517,fitness app,positive
518,best,positive
519,sup,positive


In [37]:
sentiment_mapping = {'positive': 'Positive', 'negative': 'Negative', 'neutral': 'Neutral'}
filtered_df['sentiment'] = filtered_df['sentiment'].replace(sentiment_mapping)
merged_df['Label']=merged_df['Label'].replace(sentiment_mapping)

In [38]:
# true labels
y_true = filtered_df['sentiment']

# predicted labels
#y_pred = result
y_pred=merged_df['Label']

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[ 72   0  14]
 [ 50   3 132]
 [ 35   0 215]]
Classification Report:
              precision    recall  f1-score   support

    Negative       0.46      0.84      0.59        86
     Neutral       1.00      0.02      0.03       185
    Positive       0.60      0.86      0.70       250

    accuracy                           0.56       521
   macro avg       0.68      0.57      0.44       521
weighted avg       0.72      0.56      0.45       521



In [39]:
model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

sentiment_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path)
result1 =[]
for l in df['cleaned_text']:
  result1.append(sentiment_classifier(l))

result1

[[{'label': 'Neutral', 'score': 0.9894635677337646}],
 [{'label': 'Neutral', 'score': 0.9844516515731812}],
 [{'label': 'Neutral', 'score': 0.9910281300544739}],
 [{'label': 'Neutral', 'score': 0.9887531995773315}],
 [{'label': 'Negative', 'score': 0.985499382019043}],
 [{'label': 'Neutral', 'score': 0.9704256653785706}],
 [{'label': 'Neutral', 'score': 0.9760017395019531}],
 [{'label': 'Neutral', 'score': 0.9825902581214905}],
 [{'label': 'Neutral', 'score': 0.9602766633033752}],
 [{'label': 'Neutral', 'score': 0.9825902581214905}],
 [{'label': 'Neutral', 'score': 0.9018758535385132}],
 [{'label': 'Neutral', 'score': 0.9884963035583496}],
 [{'label': 'Positive', 'score': 0.9899909496307373}],
 [{'label': 'Neutral', 'score': 0.983797013759613}],
 [{'label': 'Positive', 'score': 0.9398313760757446}],
 [{'label': 'Positive', 'score': 0.9916009902954102}],
 [{'label': 'Neutral', 'score': 0.9913439750671387}],
 [{'label': 'Neutral', 'score': 0.8778345584869385}],
 [{'label': 'Neutral', 'sc

In [40]:
labels_sent = [d['label'] for lst2 in result1 for d in lst2]
df_senti_model=pd.DataFrame(labels_sent)
df_senti_model[0:]

,0
0,Neutral
1,Neutral
2,Neutral
3,Neutral
4,Negative
...,...
516,Neutral
517,Neutral
518,Positive
519,Neutral


In [41]:
y_true = filtered_df['sentiment']
result_for_conf = df_senti_model.assign(C=y_true)
df_confusion1 = result_for_conf.applymap(lambda s: s.lower() if type(s) == str else s)
df_confusion1

,0,C
0,neutral,neutral
1,neutral,negative
2,neutral,neutral
3,neutral,negative
4,negative,negative
...,...,...
516,neutral,neutral
517,neutral,neutral
518,positive,positive
519,neutral,positive


In [42]:
# true labels
y_true = df_confusion1.iloc[:,1]
# predicted labels
y_pred = df_confusion1.iloc[:,0]

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[ 17  66   3]
 [  0 184   1]
 [  8 151  91]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.68      0.20      0.31        86
     neutral       0.46      0.99      0.63       185
    positive       0.96      0.36      0.53       250

    accuracy                           0.56       521
   macro avg       0.70      0.52      0.49       521
weighted avg       0.73      0.56      0.53       521



# **SB10K Dataset**

In [43]:
df_combined = pd.read_csv("/content/drive/MyDrive/Data Set/corpus_v1.0.cgsa .tsv", delimiter='\t')
df_combined.columns =['id', 'sentiment','text']
df_combined

,id,sentiment,text
0,367189542857482240,neutral,Tainted Talents (Ateliertagebuch.) » Wir sind ...
1,368327046574776321,neutral,Aber wenigstens kommt #Supernatural heute mal ...
2,368309870673793024,neutral,DARLEHEN - Angebot für Schufa-freie Darlehen: ...
3,362896018389475328,neutral,ANRUF ERWÜNSCHT: Hardcore Teeny Vicky Carrera:...
4,367912309303148545,neutral,Na? Wo sind Frankens heimliche Talente? - Die ...
...,...,...,...
7470,365216013760991234,neutral,"CL-Qualifikation: Zenit mit Kantersieg, Austri..."
7471,373714852122202112,negative,@AstridSchiebs Okay. @DerAtrox Machse einfach!...
7472,376593709280034816,neutral,Öffentliche Pfeifturmbegehung Führung um 11:30...
7473,377180450500136961,positive,"Ich muss gerade irgendwem, der mich nicht dafü..."


In [44]:
df = df_combined.fillna("")
df['cleaned_text'] =df['text'].apply(clean_text)
df['cleaned_text']

0               tainted talent ateliertagebuch wir allein
1       aber wenigst kommt supernatural heut mal uhr s...
2       darleh angebot schufa freie darleh gunstig anb...
3       anruf erwunscht hardcor teeny vicky carrera ha...
4       na wo frank heimlich talent die erst bewerb ta...
                              ...                        
7470    cl qualifikation zenit kantersieg austria zittert
7471          astridschieb okay deratrox machs einfach eu
7472    offent pfeifturmbegeh fuhrung uhr alt rathaus ...
7473    ich gerad irgendw dafur judged mitteil franzos...
7474      frauflausch ich mag echt gern hast fein gemacht
Name: cleaned_text, Length: 7475, dtype: object

In [45]:
model = SentimentModel()
texts = df['cleaned_text'].iloc[0:1000]
result=model.predict_sentiment(texts)

In [46]:
merged_df = pd.DataFrame(columns = ['Text'])
merged_df['Text'] = df['cleaned_text'].iloc[0:1000]
merged_df['Label'] = result
merged_df

,Text,Label
0,tainted talent ateliertagebuch wir allein,neutral
1,aber wenigst kommt supernatural heut mal uhr s...,negative
2,darleh angebot schufa freie darleh gunstig anb...,neutral
3,anruf erwunscht hardcor teeny vicky carrera ha...,neutral
4,na wo frank heimlich talent die erst bewerb ta...,neutral
...,...,...
995,cros kindermus braucht,negative
996,zdfsport extr viel braunschweig gut spiel dfb ...,neutral
997,moin moin trotz reg wunsch happy castl monday,positive
998,geisthoernch newelmedia tja iran weiss gut eig...,negative


In [47]:
# true labels
y_true = df_combined['sentiment'].iloc[0:1000]

# predicted labels
#y_pred = result
y_pred=merged_df['Label'].iloc[0:1000]

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[124  21  16]
 [240 301  65]
 [ 99  45  89]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.27      0.77      0.40       161
     neutral       0.82      0.50      0.62       606
    positive       0.52      0.38      0.44       233

    accuracy                           0.51      1000
   macro avg       0.54      0.55      0.49      1000
weighted avg       0.66      0.51      0.54      1000



In [48]:
model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

sentiment_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path)
result1 =[]
for l in df['cleaned_text'].iloc[0:1000]:
  result1.append(sentiment_classifier(l))

result1

[[{'label': 'Neutral', 'score': 0.9870354533195496}],
 [{'label': 'Neutral', 'score': 0.9874310493469238}],
 [{'label': 'Neutral', 'score': 0.9912841320037842}],
 [{'label': 'Neutral', 'score': 0.9893052577972412}],
 [{'label': 'Neutral', 'score': 0.9891679286956787}],
 [{'label': 'Neutral', 'score': 0.9917929768562317}],
 [{'label': 'Neutral', 'score': 0.9810413718223572}],
 [{'label': 'Neutral', 'score': 0.9912874102592468}],
 [{'label': 'Neutral', 'score': 0.9887914657592773}],
 [{'label': 'Neutral', 'score': 0.9892557859420776}],
 [{'label': 'Neutral', 'score': 0.9905577898025513}],
 [{'label': 'Neutral', 'score': 0.9891772866249084}],
 [{'label': 'Neutral', 'score': 0.990043044090271}],
 [{'label': 'Neutral', 'score': 0.9912017583847046}],
 [{'label': 'Neutral', 'score': 0.9912264943122864}],
 [{'label': 'Neutral', 'score': 0.9813839793205261}],
 [{'label': 'Neutral', 'score': 0.9900395274162292}],
 [{'label': 'Neutral', 'score': 0.9123337864875793}],
 [{'label': 'Neutral', 'score

In [49]:
labels_sent = [d['label'] for lst2 in result1 for d in lst2]
df_senti_model=pd.DataFrame(labels_sent)
df_senti_model[0:]

,0
0,Neutral
1,Neutral
2,Neutral
3,Neutral
4,Neutral
...,...
995,Neutral
996,Neutral
997,Neutral
998,Neutral


In [50]:
y_true = df_combined['sentiment']
result_for_conf = df_senti_model.assign(C=y_true)
df_confusion1 = result_for_conf.applymap(lambda s: s.lower() if type(s) == str else s)
df_confusion1

,0,C
0,neutral,neutral
1,neutral,neutral
2,neutral,neutral
3,neutral,neutral
4,neutral,neutral
...,...,...
995,neutral,negative
996,neutral,positive
997,neutral,positive
998,neutral,negative


In [51]:
# true labels
y_true = df_confusion1.iloc[:,1]
# predicted labels
y_pred = df_confusion1.iloc[:,0]

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[ 32 124   5]
 [  8 587  11]
 [  8 192  33]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.67      0.20      0.31       161
     neutral       0.65      0.97      0.78       606
    positive       0.67      0.14      0.23       233

    accuracy                           0.65      1000
   macro avg       0.66      0.44      0.44      1000
weighted avg       0.66      0.65      0.58      1000



# **PotTS DataSet**

In [52]:
df_combined = pd.read_csv("/content/drive/MyDrive/Data Set/7.addition.tsv", delimiter='\t')
df_combined.columns =['id','sentiment', 'text','duplicate_text','f3', 'f4','f5']
df_combined

,id,sentiment,text,duplicate_text,f3,f4,f5
0,323861158585053184,positive,ja mach das mal ich will unbedingt einen %PosS...,ja machen die mal ich wollen unbedingt eine %P...,ADV VVIMP PDS ADV PPER VMFIN ADJD ART ITJ PPER...,3/MO 3/NK 6/SB 6/MO 6/SB 0/-- 6/MO 2/OA 11/DM ...,Sentiment::markable_2::Sentiment=True pers=3|n...
1,328111953920806913,positive,Rating - Update bei Cache me if you can ! %Pos...,Rating - Update bei Cache me if you can ! %Pos...,NN $( NN APPR NE ADJA FM FM FM $. ITJ APPR NE ...,6/SB 1/-- 1/CJ 3/MNR 4/NK 0/-- 9/PNC 9/PNC 6/M...,case=nom|gender=neut|num=sg _ case=acc|gender=...
2,323759724241764352,positive,Omg ich komm von der Schule und sehe dass %Use...,Omg ich kommen von die Schule und sehen dass %...,NE PPER VVIMP APPR ART NN KON VVFIN KOUS NE PR...,3/UC 1/SB 0/-- 3/MO 6/NK 4/NK 3/CD 7/CJ 13/CP ...,mood=ind|EmoExpression::markable_100500263::Em...
3,323839282949287936,positive,ein tag voller adrenalin und kalorien %PosSmil...,ein tagen voll Adrenalin und Kalorie %PosSmile...,PTKVZ VVIMP ADJA NN KON NN ITJ XY,8/NK 1/NK 4/NK 2/OA 4/CD 5/CJ 1/NK 0/--,Target::markable_14::Target=True|Target::marka...
4,323720767596531712,positive,Das war vielleicht ein Wochenende %PosSmiley D...,die sein vielleicht eine Wochenende %PosSmiley...,PDS VAFIN ADV ART NN ITJ ART NE NN VVFIN APPRA...,2/SB 0/-- 2/MO 5/NK 2/PD 8/NK 8/NK 5/NK 10/SB ...,case=nom|Sentiment::markable_17::Sentiment=Tru...
...,...,...,...,...,...,...,...
93,328256313593323521,positive,.. ich ihn retweeten kann. aber das würde eh n...,.. ich er retweeten kann. aber die werden eh n...,NN PPER PPER NN NE ADV PDS VAFIN ADV PTKNEG VV...,0/-- 4/SB 11/OA 7/RE 4/NK 7/MO 8/SB 0/-- 11/MO...,case=nom|gender=fem|num=sg case=nom|*=True|num...
94,322318494773424128,positive,Fertigteilhaus mit Vollsystemkeller . Mit Wärm...,Fertigteilhaus mit Vollsystemkeller . mit Wärm...,NN APPR NN $. APPR NN $. NN $. NN $. XY,0/-- 1/MNR 2/NK 3/-- 0/-- 5/NK 6/-- 0/-- 8/-- ...,case=nom|gender=neut|num=sg _ case=dat|gender=...
95,323575746721361920,positive,"Während Tony so abgeht , esse ich gerade Früch...","während Tony so abgehen , essen ich gerade Frü...","APPR NE ADV VVFIN $, VVFIN PPER ADV NN $.",4/CP 4/SB 4/MO 6/MO 4/-- 0/-- 6/SB 6/MO 6/OA 9/--,_ case=dat|*=True|num=sg _ pers=3|num=sg|tense...
96,322242804791275520,negative,"Hach , mal schauen . Ich bin so unmotiviert .","Hach , mal schauen . ich sein so unmotiviert .","NN $, ADV VVINF $. PPER VAFIN ADV ADJD $.",0/-- 1/-- 4/MO 1/CJ 4/-- 7/SB 0/-- 9/MO 7/PD 9/--,case=nom|gender=masc|num=sg _ _ _ _ case=nom|S...


In [53]:
df = df_combined.fillna("")
df['cleaned_text'] =df['text'].apply(clean_text)
df['cleaned_text']

0     ja mach mal unbedingt possmiley weisst garnich...
1     rating updat cach me if you can possmiley patc...
2     omg komm schul seh user geschrieb possmiley po...
3              tag voll adrenalin kalori possmiley link
4     das vielleicht wochen possmiley der spd bundes...
                            ...                        
93                           retweet eh klapp possmiley
94    fertigteilhaus vollsystemkell mit warmeruckgew...
95           wahrend tony abgeht ess gerad fruchtetraum
96                       hach mal schau ich unmotiviert
97                        komm bild possmiley possmiley
Name: cleaned_text, Length: 98, dtype: object

In [56]:
from germansentiment import SentimentModel
model = SentimentModel()
texts = df['cleaned_text']
result=model.predict_sentiment(texts)

In [57]:
merged_df = pd.DataFrame(columns = ['Text'])
merged_df['Text'] = df['cleaned_text']
merged_df['Label'] = result
merged_df

,Text,Label
0,ja mach mal unbedingt possmiley weisst garnich...,negative
1,rating updat cach me if you can possmiley patc...,negative
2,omg komm schul seh user geschrieb possmiley po...,negative
3,tag voll adrenalin kalori possmiley link,negative
4,das vielleicht wochen possmiley der spd bundes...,neutral
...,...,...
93,retweet eh klapp possmiley,negative
94,fertigteilhaus vollsystemkell mit warmeruckgew...,neutral
95,wahrend tony abgeht ess gerad fruchtetraum,neutral
96,hach mal schau ich unmotiviert,negative


In [58]:
# true labels
y_true = df_combined['sentiment']

# predicted labels
#y_pred = result
y_pred=merged_df['Label']

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[ 0  4  0  0]
 [ 0 16  7  0]
 [ 0 14 10  2]
 [ 0 28 11  6]]
Classification Report:
              precision    recall  f1-score   support

       mixed       0.00      0.00      0.00         4
    negative       0.26      0.70      0.38        23
     neutral       0.36      0.38      0.37        26
    positive       0.75      0.13      0.23        45

    accuracy                           0.33        98
   macro avg       0.34      0.30      0.24        98
weighted avg       0.50      0.33      0.29        98



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

sentiment_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path)
result1 =[]
for l in df['cleaned_text'].iloc[0:1000]:
  result1.append(sentiment_classifier(l))

result1

[[{'label': 'Neutral', 'score': 0.9837431311607361}],
 [{'label': 'Neutral', 'score': 0.9890469908714294}],
 [{'label': 'Neutral', 'score': 0.9889116287231445}],
 [{'label': 'Neutral', 'score': 0.9853090643882751}],
 [{'label': 'Neutral', 'score': 0.9872375726699829}],
 [{'label': 'Neutral', 'score': 0.9901372194290161}],
 [{'label': 'Neutral', 'score': 0.982848048210144}],
 [{'label': 'Neutral', 'score': 0.9863303899765015}],
 [{'label': 'Neutral', 'score': 0.9887625575065613}],
 [{'label': 'Neutral', 'score': 0.9836428761482239}],
 [{'label': 'Neutral', 'score': 0.9701022505760193}],
 [{'label': 'Neutral', 'score': 0.9858967661857605}],
 [{'label': 'Neutral', 'score': 0.9889802932739258}],
 [{'label': 'Neutral', 'score': 0.9882305860519409}],
 [{'label': 'Neutral', 'score': 0.9623160362243652}],
 [{'label': 'Neutral', 'score': 0.9869834184646606}],
 [{'label': 'Neutral', 'score': 0.991054117679596}],
 [{'label': 'Neutral', 'score': 0.9919026494026184}],
 [{'label': 'Neutral', 'score'

In [60]:
labels_sent = [d['label'] for lst2 in result1 for d in lst2]
df_senti_model=pd.DataFrame(labels_sent)
df_senti_model[0:]

,0
0,Neutral
1,Neutral
2,Neutral
3,Neutral
4,Neutral
...,...
93,Neutral
94,Neutral
95,Neutral
96,Neutral


In [61]:

y_true = df_combined['sentiment']
result_for_conf = df_senti_model.assign(C=y_true)
df_confusion1 = result_for_conf.applymap(lambda s: s.lower() if type(s) == str else s)
df_confusion1


,0,C
0,neutral,positive
1,neutral,positive
2,neutral,positive
3,neutral,positive
4,neutral,positive
...,...,...
93,neutral,positive
94,neutral,positive
95,neutral,positive
96,neutral,negative


In [62]:
# true labels
y_true = df_confusion1.iloc[:,1]
# predicted labels
y_pred = df_confusion1.iloc[:,0]

# compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:")
print(cm)

# compute classification report
cr = classification_report(y_true, y_pred)

# print classification report
print("Classification Report:")
print(cr)

Confusion Matrix:
[[ 0  0  4  0]
 [ 0  2 21  0]
 [ 0  0 26  0]
 [ 0  0 44  1]]
Classification Report:
              precision    recall  f1-score   support

       mixed       0.00      0.00      0.00         4
    negative       1.00      0.09      0.16        23
     neutral       0.27      1.00      0.43        26
    positive       1.00      0.02      0.04        45

    accuracy                           0.30        98
   macro avg       0.57      0.28      0.16        98
weighted avg       0.77      0.30      0.17        98



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
